In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
print("hi")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import os

hi


2025-06-18 11:47:23.600301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750247243.831917      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750247243.897313      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# !pip install transformers==4.28.0 torch pandas numpy scikit-learn --quiet

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import os

# Step 1: Load and preprocess the dataset
def load_data(file_path):
    # Load CSV file
    df = pd.read_csv(file_path)
    
    # Ensure expected columns
    if 'article' not in df.columns or 'Bias_Label' not in df.columns:
        raise ValueError("Dataset must have 'article' and 'Bias_Label' columns")
    
    # Map string labels to integers
    label_map = {'Neutral': 0, 'Positive Bias': 1, 'Negative Bias': 2}
    df['label'] = df['Bias_Label'].map(label_map)
    
    # Check for invalid labels
    if df['label'].isna().any():
        raise ValueError("Some Bias_Label values are not 'Positive', 'Negative', or 'Neutral'")
    
    # Remove rows with missing articles
    df = df.dropna(subset=['article'])
    
    # Print class distribution
    print("Class Distribution:")
    print(df['Bias_Label'].value_counts())
    
    return df['article'].values, df['label'].values

# Step 2: Create a custom dataset class for BERT
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):  # Reduced max_length for larger dataset
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize text
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.texts)

# Step 3: Main function to fine-tune BERT
def fine_tune_bert(file_path='/kaggle/input/data-set/your file name_bias_LABEL.csv', output_dir='/kaggle/working/bias_model', num_epochs=11, batch_size=8):
    # *** Epochs and Batch Size Defined Here ***
    print(f"Using {num_epochs} epochs and batch size of {batch_size} (effective batch size={batch_size*2} with gradient accumulation)")
    
    # Load data
    texts, labels = load_data(file_path)
    
    # Split data into train (80%), validation (10%), and test (10%) sets
    train_texts, temp_texts, train_labels, temp_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42, stratify=labels
    )
    val_texts, test_texts, val_labels, test_labels = train_test_split(
        temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
    )
    
    # Print split sizes
    print(f"Train set: {len(train_texts)} articles (~{len(train_texts)/6500*100:.1f}%)")
    print(f"Validation set: {len(val_texts)} articles (~{len(val_texts)/6500*100:.1f}%)")
    print(f"Test set: {len(test_texts)} articles (~{len(test_texts)/6500*100:.1f}%)")
    
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    # Create datasets
    train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
    val_dataset = NewsDataset(val_texts, val_labels, tokenizer)
    test_dataset = NewsDataset(test_texts, test_labels, tokenizer)
    
    # Initialize model
    # Note: Warning about uninitialized weights is normal; training below will initialize them
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=3
    )
    
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,  # *** Epochs Set Here (3) ***
        per_device_train_batch_size=batch_size,  # *** Batch Size Set Here (8) ***
        per_device_eval_batch_size=batch_size,  # *** Batch Size for Evaluation (8) ***
        gradient_accumulation_steps=2,  # Effective batch size=16, reduces memory usage
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='steps',
        save_steps=300,  # Multiple of eval_steps (3 * 100), saves ~once per epoch
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        logging_dir='/kaggle/working/logs',
        logging_steps=50,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),  # Mixed precision for GPU
        report_to='none'  # Disable WandB
    )
    
    # Define compute_metrics function for evaluation
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        report = classification_report(labels, predictions, output_dict=True, zero_division=0)
        return {
            'accuracy': report['accuracy'],
            'precision': report['weighted avg']['precision'],
            'recall': report['weighted avg']['recall'],
            'f1': report['weighted avg']['f1-score']
        }
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    
    # Train the model (initializes weights, resolving the warning)
    trainer.train()
    
    # Evaluate on test set
    test_results = trainer.evaluate(test_dataset)
    print("Test Set Evaluation:", test_results)
    
    # Save the model and tokenizer
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model and tokenizer saved to {output_dir}")
    
    return model, tokenizer, test_results

# Step 4: Prediction function for new articles
def predict_bias(text, model, tokenizer):
    labels = ['Neutral', 'Positive', 'Negative']
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()[0]  # Move to CPU before NumPy conversion
    label = labels[probs.argmax()]
    return label, probs










In [6]:
model, tokenizer, test_results=fine_tune_bert()

Using 11 epochs and batch size of 8 (effective batch size=16 with gradient accumulation)
Class Distribution:
Bias_Label
Positive Bias    3805
Negative Bias    2651
Neutral            34
Name: count, dtype: int64
Train set: 5192 articles (~79.9%)
Validation set: 649 articles (~10.0%)
Test set: 649 articles (~10.0%)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.440700,0.397608,0.835131,0.830746,0.835131,0.832795
200,0.401800,0.440303,0.824345,0.820610,0.824345,0.822472
300,0.285800,0.454777,0.828968,0.824706,0.828968,0.826776
400,0.157900,0.530244,0.818182,0.814478,0.818182,0.816325
500,0.157200,0.671615,0.821263,0.818268,0.821263,0.819673
600,0.179300,0.886762,0.819723,0.815259,0.819723,0.816485
700,0.081800,1.828474,0.805855,0.815636,0.805855,0.805470
800,0.206300,2.328912,0.822804,0.818945,0.822804,0.818993
900,0.119800,2.773522,0.807396,0.806445,0.807396,0.806315
1000,0.122200,2.886800,0.810478,0.807428,0.810478,0.808886


Test Set Evaluation: {'eval_loss': 2.7148425579071045, 'eval_accuracy': 0.8289676425269645, 'eval_precision': 0.8315285658829207, 'eval_recall': 0.8289676425269645, 'eval_f1': 0.8302460644437978, 'eval_runtime': 10.8379, 'eval_samples_per_second': 59.882, 'eval_steps_per_second': 3.783, 'epoch': 10.935384615384615}
Model and tokenizer saved to /kaggle/working/bias_model


In [16]:
text="we went to manali and find that it has shortage of water."

predict_bias(text,model,tokenizer)


('Negative',
 array([1.0110099e-04, 9.1619659e-03, 9.9073690e-01], dtype=float32))

In [ ]:
# text translation model fine tuning


In [17]:
import pandas as pd
import torch
from transformers import MarianTokenizer, MarianMTModel, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset_path = "/kaggle/input/dataset-tt/shortened_hindi_engish.csv"
df = pd.read_csv(dataset_path)
# Ensure the dataset has the correct columns
assert 'english_sentence' in df.columns and 'hindi_sentence' in df.columns, "Dataset must have 'english_sentence' and 'hindi_sentence' columns"

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['english_sentence', 'hindi_sentence']])

# Split dataset into train and validation sets (90% train, 10% validation)
train_dataset, eval_dataset = train_test_split(dataset.to_pandas(), test_size=0.1, random_state=42)
train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

# Load tokenizer and model
model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Preprocess function to tokenize the data
def preprocess_function(examples):
    inputs = [f"translate English to Hindi: {text}" for text in examples["english_sentence"]]
    targets = examples["hindi_sentence"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/marianmt_en_hi_finetuned",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/kaggle/working/logs",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none",  # Disable wandb logging
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained("/kaggle/working/marianmt_en_hi_finetuned")
tokenizer.save_pretrained("/kaggle/working/marianmt_en_hi_finetuned")

# Example inference
def translate(text):
    inputs = tokenizer(f"translate English to Hindi: {text}", return_tensors="pt", padding=True).to(device)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Test the model
sample_text = "Hello, how are you?"
translated_text = translate(sample_text)
print(f"Input: {sample_text}")
print(f"Translated: {translated_text}")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

Map:   0%|          | 0/29346 [00:00<?, ? examples/s]

Map:   0%|          | 0/3261 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,0.739300,0.723182
1000,0.678300,0.674365
1500,0.628400,0.650147
2000,0.556500,0.638122
2500,0.585700,0.624839
3000,0.531400,0.620131
3500,0.513800,0.612858
4000,0.490400,0.610963
4500,0.466800,0.605601
5000,0.459500,0.606703


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Input: Hello, how are you?
Translated: आप कैसे हैं?


In [20]:
translate

'क्या आप रोज़ाना श्रेणी के व्याख्यानों में जाते हैं ?'